<h3><b>ABOUT DATASET</b></h3>

The dataset contains a subset of locations and attributes of incidents reported in the ASAP (Analytical Services Application) crime report database by the District of Columbia Metropolitan Police Department (MPD). Visit crimecards.dc.gov for more information. This data is shared via an automated process where addresses are geocoded to the District's Master Address Repository and assigned to the appropriate street block. Block locations for some crime points could not be automatically assigned resulting in 0,0 for x,y coordinates. These can be interactively assigned using the MAR Geocoder.On February 1 2020, the methodology of geography assignments of crime data was modified to increase accuracy. From January 1 2020 going forward, all crime data will have Ward, ANC, SMD, BID, Neighborhood Cluster, Voting Precinct, Block Group and Census Tract values calculated prior to, rather than after, anonymization to the block level. This change impacts approximately one percent of Ward assignments.

Data source: https://catalog.data.gov/dataset/crime-incidents-in-2025

<h3><b>DATASET</b></h3>

In [24]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

df = pd.read_csv('crime_incidents_dataset.csv')
df.head(3)

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,400046.41,137753.65,25017503,2025/02/05 19:50:58+00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.41,137753.65,...,008803 2,8803.0,Precinct 76,38.907642,-76.999465,NaN,2025/02/05 19:22:00+00,2025/02/05 19:25:00+00,685784661,NaN
1,397868.68,141140.38,25013087,2025/01/28 12:08:29+00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.68,141140.38,...,002503 2,2503.0,Precinct 47,38.938148,-77.024583,NaN,2025/01/28 11:06:00+00,2025/01/28 11:07:00+00,685784675,NaN
2,399489.48,137372.65,25012701,2025/01/27 17:28:06+00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.48,137372.65,...,010603 3,10603.0,Precinct 144,38.904210,-77.005886,NOMA,2025/01/27 15:00:00+00,2025/01/27 15:20:00+00,685784796,NaN


<h3><b>DATA CLEANING: PART 1</b></h3> 

Handling missing value

In [25]:
df.isnull().sum()

X                          0
Y                          0
CCN                        0
REPORT_DAT                 0
SHIFT                      0
METHOD                     0
OFFENSE                    0
BLOCK                      0
XBLOCK                     0
YBLOCK                     0
WARD                       1
ANC                        1
DISTRICT                   8
PSA                        8
NEIGHBORHOOD_CLUSTER       1
BLOCK_GROUP                4
CENSUS_TRACT               4
VOTING_PRECINCT            1
LATITUDE                   0
LONGITUDE                  0
BID                     3429
START_DATE                 0
END_DATE                 303
OBJECTID                   0
OCTO_RECORD_ID          4164
dtype: int64

Drop: OCTO_RECORD_ID and BID

In [26]:
df.drop(columns=['BID'], inplace=True)
df.drop(columns=['OCTO_RECORD_ID'], inplace=True)

In [27]:
df.isnull().sum()

X                         0
Y                         0
CCN                       0
REPORT_DAT                0
SHIFT                     0
METHOD                    0
OFFENSE                   0
BLOCK                     0
XBLOCK                    0
YBLOCK                    0
WARD                      1
ANC                       1
DISTRICT                  8
PSA                       8
NEIGHBORHOOD_CLUSTER      1
BLOCK_GROUP               4
CENSUS_TRACT              4
VOTING_PRECINCT           1
LATITUDE                  0
LONGITUDE                 0
START_DATE                0
END_DATE                303
OBJECTID                  0
dtype: int64

Filling missing value in WARD and ANC 

In [28]:
missing_ward = df[df['WARD'].isnull()].head(1)
print(missing_ward[['WARD']])

      WARD
2157   NaN


In [29]:
missing_ward = df[df['WARD'].isnull()]
print(missing_ward[['ANC']])


      ANC
2157  NaN


In [30]:
print(missing_ward[['DISTRICT']])

      DISTRICT
2157       5.0


In [31]:
print(df.groupby('DISTRICT')['WARD'].nunique().sort_values())
print(df.groupby('PSA')['WARD'].nunique().sort_values())  


DISTRICT
7.0    1
6.0    2
2.0    3
4.0    3
5.0    3
1.0    4
3.0    4
Name: WARD, dtype: int64
PSA
104.0    1
105.0    1
206.0    1
207.0    1
205.0    1
204.0    1
203.0    1
202.0    1
208.0    1
402.0    1
403.0    1
404.0    1
304.0    1
303.0    1
302.0    1
209.0    1
407.0    1
401.0    1
702.0    1
703.0    1
704.0    1
604.0    1
602.0    1
605.0    1
601.0    1
506.0    1
504.0    1
505.0    1
503.0    1
502.0    1
409.0    1
408.0    1
708.0    1
705.0    1
706.0    1
707.0    1
701.0    1
608.0    1
301.0    2
305.0    2
101.0    2
107.0    2
108.0    2
103.0    2
106.0    2
102.0    2
307.0    2
201.0    2
607.0    2
606.0    2
405.0    2
406.0    2
501.0    2
603.0    2
507.0    2
306.0    3
308.0    3
Name: WARD, dtype: int64


In [32]:
df['WARD'] = df['WARD'].fillna(df.groupby('DISTRICT')['WARD'].transform(lambda x: x.mode()[0] if not x.mode().empty else None))
df['ANC'] = df['ANC'].fillna(df.groupby('DISTRICT')['ANC'].transform(lambda x: x.mode()[0] if not x.mode().empty else None))


In [33]:
df.isnull().sum()

X                         0
Y                         0
CCN                       0
REPORT_DAT                0
SHIFT                     0
METHOD                    0
OFFENSE                   0
BLOCK                     0
XBLOCK                    0
YBLOCK                    0
WARD                      0
ANC                       0
DISTRICT                  8
PSA                       8
NEIGHBORHOOD_CLUSTER      1
BLOCK_GROUP               4
CENSUS_TRACT              4
VOTING_PRECINCT           1
LATITUDE                  0
LONGITUDE                 0
START_DATE                0
END_DATE                303
OBJECTID                  0
dtype: int64

In [34]:
print(df['WARD'].loc[2157])

5.0


In [35]:
print(df['ANC'].loc[2157])

5D


Saving data cleaning part 1 result

In [36]:
df.to_csv('data_crime_cleaned_1.csv', index=False)

<h3><b>DATA CLEANING: PART 2</b></h3> 

Handling missing value

In [37]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

df_new = pd.read_csv('data_crime_cleaned_1.csv')
df_new.head(4)

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,PSA,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,START_DATE,END_DATE,OBJECTID
0,400046.41,137753.65,25017503,2025/02/05 19:50:58+00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.41,137753.65,...,501.0,Cluster 23,008803 2,8803.0,Precinct 76,38.907642,-76.999465,2025/02/05 19:22:00+00,2025/02/05 19:25:00+00,685784661
1,397868.68,141140.38,25013087,2025/01/28 12:08:29+00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.68,141140.38,...,404.0,Cluster 18,002503 2,2503.0,Precinct 47,38.938148,-77.024583,2025/01/28 11:06:00+00,2025/01/28 11:07:00+00,685784675
2,399489.48,137372.65,25012701,2025/01/27 17:28:06+00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.48,137372.65,...,501.0,Cluster 25,010603 3,10603.0,Precinct 144,38.904210,-77.005886,2025/01/27 15:00:00+00,2025/01/27 15:20:00+00,685784796
3,401498.97,138734.57,25008887,2025/01/19 19:59:51+00,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF NEW YORK AVENUE NE,401498.97,138734.57,...,506.0,Cluster 23,008803 1,8803.0,Precinct 76,38.916477,-76.982716,2025/01/19 19:22:00+00,2025/01/19 19:32:00+00,685784829


In [38]:
df_new.isnull().sum()

X                         0
Y                         0
CCN                       0
REPORT_DAT                0
SHIFT                     0
METHOD                    0
OFFENSE                   0
BLOCK                     0
XBLOCK                    0
YBLOCK                    0
WARD                      0
ANC                       0
DISTRICT                  8
PSA                       8
NEIGHBORHOOD_CLUSTER      1
BLOCK_GROUP               4
CENSUS_TRACT              4
VOTING_PRECINCT           1
LATITUDE                  0
LONGITUDE                 0
START_DATE                0
END_DATE                303
OBJECTID                  0
dtype: int64

Missing value handling in DISTRICT and PSA

In [39]:
missing_district = df_new[df_new['DISTRICT'].isnull()].head(10)
print(missing_district[['DISTRICT']])

      DISTRICT
383        NaN
523        NaN
1550       NaN
1555       NaN
2598       NaN
2627       NaN
2789       NaN
3125       NaN


In [40]:
missing_district = df_new[df_new['PSA'].isnull()].head(10)
print(missing_district[['PSA']])

      PSA
383   NaN
523   NaN
1550  NaN
1555  NaN
2598  NaN
2627  NaN
2789  NaN
3125  NaN


In [41]:
df_new['DISTRICT'] = df_new.groupby(['WARD', 'ANC'])['DISTRICT'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))
df_new['PSA'] = df_new.groupby(['WARD', 'ANC'])['PSA'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))

In [42]:
df_new.isnull().sum()

X                         0
Y                         0
CCN                       0
REPORT_DAT                0
SHIFT                     0
METHOD                    0
OFFENSE                   0
BLOCK                     0
XBLOCK                    0
YBLOCK                    0
WARD                      0
ANC                       0
DISTRICT                  0
PSA                       0
NEIGHBORHOOD_CLUSTER      1
BLOCK_GROUP               4
CENSUS_TRACT              4
VOTING_PRECINCT           1
LATITUDE                  0
LONGITUDE                 0
START_DATE                0
END_DATE                303
OBJECTID                  0
dtype: int64

Handling Missing value handling in NEIGHBORHOOD_CLUSTER

In [43]:
missing_neighborhood_cluster = df_new[df_new['NEIGHBORHOOD_CLUSTER'].isnull()].head(10)
print(missing_neighborhood_cluster[['NEIGHBORHOOD_CLUSTER']])

     NEIGHBORHOOD_CLUSTER
2157                  NaN


In [44]:
df_new['NEIGHBORHOOD_CLUSTER'].fillna(df_new['NEIGHBORHOOD_CLUSTER'].mode()[0], inplace=True)

C:\Users\LOQ\AppData\Local\Temp\ipykernel_6204\2760611688.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_new['NEIGHBORHOOD_CLUSTER'].fillna(df_new['NEIGHBORHOOD_CLUSTER'].mode()[0], inplace=True)


In [45]:
df_new.isnull().sum()

X                         0
Y                         0
CCN                       0
REPORT_DAT                0
SHIFT                     0
METHOD                    0
OFFENSE                   0
BLOCK                     0
XBLOCK                    0
YBLOCK                    0
WARD                      0
ANC                       0
DISTRICT                  0
PSA                       0
NEIGHBORHOOD_CLUSTER      0
BLOCK_GROUP               4
CENSUS_TRACT              4
VOTING_PRECINCT           1
LATITUDE                  0
LONGITUDE                 0
START_DATE                0
END_DATE                303
OBJECTID                  0
dtype: int64

Missing value handling in BLOCK_GROUP and CENCUS_TRACT

In [46]:
missing_block_group = df_new[df_new['BLOCK_GROUP'].isnull()].head(10)
print(missing_block_group[['BLOCK_GROUP']])

     BLOCK_GROUP
734          NaN
2157         NaN
2454         NaN
3389         NaN


In [47]:
missing_cencus_tract = df_new[df_new['CENSUS_TRACT'].isnull()].head(10)
print(missing_cencus_tract[['CENSUS_TRACT']])

      CENSUS_TRACT
734            NaN
2157           NaN
2454           NaN
3389           NaN


In [48]:
df_new['BLOCK_GROUP'] = df_new.groupby(['WARD', 'ANC', 'DISTRICT', 'PSA'])['BLOCK_GROUP'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))
df_new['CENSUS_TRACT'] = df_new.groupby(['WARD', 'ANC', 'DISTRICT', 'PSA'])['CENSUS_TRACT'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))

C:\Users\LOQ\AppData\Local\Temp\ipykernel_6204\1425372933.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new['BLOCK_GROUP'] = df_new.groupby(['WARD', 'ANC', 'DISTRICT', 'PSA'])['BLOCK_GROUP'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))


In [49]:
df_new.isnull().sum()

X                         0
Y                         0
CCN                       0
REPORT_DAT                0
SHIFT                     0
METHOD                    0
OFFENSE                   0
BLOCK                     0
XBLOCK                    0
YBLOCK                    0
WARD                      0
ANC                       0
DISTRICT                  0
PSA                       0
NEIGHBORHOOD_CLUSTER      0
BLOCK_GROUP               1
CENSUS_TRACT              1
VOTING_PRECINCT           1
LATITUDE                  0
LONGITUDE                 0
START_DATE                0
END_DATE                303
OBJECTID                  0
dtype: int64

In [50]:
missing_block_group = df_new[df_new['BLOCK_GROUP'].isnull()].head(10)
print(missing_block_group[['BLOCK_GROUP']])

     BLOCK_GROUP
2157         NaN


In [51]:
missing_cencus_tract = df_new[df_new['CENSUS_TRACT'].isnull()].head(10)
print(missing_cencus_tract[['CENSUS_TRACT']])

      CENSUS_TRACT
2157           NaN


In [52]:
df_new['BLOCK_GROUP'].fillna(df_new.groupby('NEIGHBORHOOD_CLUSTER')['BLOCK_GROUP'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else x), inplace=True)
df_new['CENSUS_TRACT'].fillna(df_new.groupby('NEIGHBORHOOD_CLUSTER')['CENSUS_TRACT'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else x), inplace=True)

C:\Users\LOQ\AppData\Local\Temp\ipykernel_6204\3786789559.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_new['BLOCK_GROUP'].fillna(df_new.groupby('NEIGHBORHOOD_CLUSTER')['BLOCK_GROUP'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else x), inplace=True)
C:\Users\LOQ\AppData\Local\Temp\ipykernel_6204\3786789559.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method wil

In [53]:
df_new.isnull().sum()

X                         0
Y                         0
CCN                       0
REPORT_DAT                0
SHIFT                     0
METHOD                    0
OFFENSE                   0
BLOCK                     0
XBLOCK                    0
YBLOCK                    0
WARD                      0
ANC                       0
DISTRICT                  0
PSA                       0
NEIGHBORHOOD_CLUSTER      0
BLOCK_GROUP               0
CENSUS_TRACT              0
VOTING_PRECINCT           1
LATITUDE                  0
LONGITUDE                 0
START_DATE                0
END_DATE                303
OBJECTID                  0
dtype: int64

Missing value handling in VOTING_PRECINCT

In [54]:
missing_voting_precinct = df_new[df_new['VOTING_PRECINCT'].isnull()].head(10)
print(missing_voting_precinct[['VOTING_PRECINCT']])

     VOTING_PRECINCT
2157             NaN


In [55]:
voting_percint_mode = df_new['VOTING_PRECINCT'].mode()[0]
df_new['VOTING_PRECINCT'].fillna(voting_percint_mode, inplace=True)

C:\Users\LOQ\AppData\Local\Temp\ipykernel_6204\2591348092.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_new['VOTING_PRECINCT'].fillna(voting_percint_mode, inplace=True)


In [56]:
df_new.isnull().sum()

X                         0
Y                         0
CCN                       0
REPORT_DAT                0
SHIFT                     0
METHOD                    0
OFFENSE                   0
BLOCK                     0
XBLOCK                    0
YBLOCK                    0
WARD                      0
ANC                       0
DISTRICT                  0
PSA                       0
NEIGHBORHOOD_CLUSTER      0
BLOCK_GROUP               0
CENSUS_TRACT              0
VOTING_PRECINCT           0
LATITUDE                  0
LONGITUDE                 0
START_DATE                0
END_DATE                303
OBJECTID                  0
dtype: int64

Fill with temp value

In [57]:
df_new['END_DATE'] = df_new['END_DATE'].fillna("1900/01/01 00:00:00+00")

In [58]:
df_new.isnull().sum()

X                       0
Y                       0
CCN                     0
REPORT_DAT              0
SHIFT                   0
METHOD                  0
OFFENSE                 0
BLOCK                   0
XBLOCK                  0
YBLOCK                  0
WARD                    0
ANC                     0
DISTRICT                0
PSA                     0
NEIGHBORHOOD_CLUSTER    0
BLOCK_GROUP             0
CENSUS_TRACT            0
VOTING_PRECINCT         0
LATITUDE                0
LONGITUDE               0
START_DATE              0
END_DATE                0
OBJECTID                0
dtype: int64

Saving data cleaning part 2 result

In [59]:
df_new.to_csv('data_crime_cleaned_2.csv', index=False)

<h3><b>DATA CLEANING: PART 3</b></h3> 

Changing data types

In [60]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

df_new_2 = pd.read_csv('data_crime_cleaned_2.csv')
df_new_2.head(4)

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,PSA,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,START_DATE,END_DATE,OBJECTID
0,400046.41,137753.65,25017503,2025/02/05 19:50:58+00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.41,137753.65,...,501.0,Cluster 23,008803 2,8803.0,Precinct 76,38.907642,-76.999465,2025/02/05 19:22:00+00,2025/02/05 19:25:00+00,685784661
1,397868.68,141140.38,25013087,2025/01/28 12:08:29+00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.68,141140.38,...,404.0,Cluster 18,002503 2,2503.0,Precinct 47,38.938148,-77.024583,2025/01/28 11:06:00+00,2025/01/28 11:07:00+00,685784675
2,399489.48,137372.65,25012701,2025/01/27 17:28:06+00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.48,137372.65,...,501.0,Cluster 25,010603 3,10603.0,Precinct 144,38.904210,-77.005886,2025/01/27 15:00:00+00,2025/01/27 15:20:00+00,685784796
3,401498.97,138734.57,25008887,2025/01/19 19:59:51+00,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF NEW YORK AVENUE NE,401498.97,138734.57,...,506.0,Cluster 23,008803 1,8803.0,Precinct 76,38.916477,-76.982716,2025/01/19 19:22:00+00,2025/01/19 19:32:00+00,685784829


In [61]:
df_new_2.isnull().sum()

X                       0
Y                       0
CCN                     0
REPORT_DAT              0
SHIFT                   0
METHOD                  0
OFFENSE                 0
BLOCK                   0
XBLOCK                  0
YBLOCK                  0
WARD                    0
ANC                     0
DISTRICT                0
PSA                     0
NEIGHBORHOOD_CLUSTER    0
BLOCK_GROUP             0
CENSUS_TRACT            0
VOTING_PRECINCT         0
LATITUDE                0
LONGITUDE               0
START_DATE              0
END_DATE                0
OBJECTID                0
dtype: int64

In [62]:
df_new_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   X                     4164 non-null   float64
 1   Y                     4164 non-null   float64
 2   CCN                   4164 non-null   int64  
 3   REPORT_DAT            4164 non-null   object 
 4   SHIFT                 4164 non-null   object 
 5   METHOD                4164 non-null   object 
 6   OFFENSE               4164 non-null   object 
 7   BLOCK                 4164 non-null   object 
 8   XBLOCK                4164 non-null   float64
 9   YBLOCK                4164 non-null   float64
 10  WARD                  4164 non-null   float64
 11  ANC                   4164 non-null   object 
 12  DISTRICT              4164 non-null   float64
 13  PSA                   4164 non-null   float64
 14  NEIGHBORHOOD_CLUSTER  4164 non-null   object 
 15  BLOCK_GROUP          

Date time columns: 'REPORT_DAT', 'START_DATE', 'END_DATE'

In [63]:
date_cols = ['REPORT_DAT', 'START_DATE', 'END_DATE']
for col in date_cols:
    df_new_2[col] = pd.to_datetime(df_new_2[col], errors='coerce')

In [64]:
df_new_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   X                     4164 non-null   float64            
 1   Y                     4164 non-null   float64            
 2   CCN                   4164 non-null   int64              
 3   REPORT_DAT            4164 non-null   datetime64[ns, UTC]
 4   SHIFT                 4164 non-null   object             
 5   METHOD                4164 non-null   object             
 6   OFFENSE               4164 non-null   object             
 7   BLOCK                 4164 non-null   object             
 8   XBLOCK                4164 non-null   float64            
 9   YBLOCK                4164 non-null   float64            
 10  WARD                  4164 non-null   float64            
 11  ANC                   4164 non-null   object             
 12  DISTRI

Categorical columns: 'SHIFT', 'METHOD', 'OFFENSE', 'ANC'

In [65]:
df_new_2.drop(columns=['VOTING_PRECINCT'], inplace=True)

In [66]:
categorical_cols = ['SHIFT', 'METHOD', 'OFFENSE', 'ANC']
for col in categorical_cols:
    df_new_2[col] = df_new_2[col].astype('category')

In [67]:
df_new_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   X                     4164 non-null   float64            
 1   Y                     4164 non-null   float64            
 2   CCN                   4164 non-null   int64              
 3   REPORT_DAT            4164 non-null   datetime64[ns, UTC]
 4   SHIFT                 4164 non-null   category           
 5   METHOD                4164 non-null   category           
 6   OFFENSE               4164 non-null   category           
 7   BLOCK                 4164 non-null   object             
 8   XBLOCK                4164 non-null   float64            
 9   YBLOCK                4164 non-null   float64            
 10  WARD                  4164 non-null   float64            
 11  ANC                   4164 non-null   category           
 12  DISTRI

Integer64 columns: 'CCN', 'WARD', 'DISTRICT', 'PSA', 'NEIGHBORHOOD_CLUSTER', 'BLOCK_GROUP', 'CENSUS_TRACT', 'OBJECTID'

In [68]:
df_new_2[['BLOCK_CODE', 'BLOCK_GROUP']] = df_new_2['BLOCK_GROUP'].str.split(' ', expand=True)

In [69]:
integer_columns = ['CCN', 'WARD', 'DISTRICT', 'PSA', 'BLOCK_GROUP', 'CENSUS_TRACT', 'OBJECTID']
for col in integer_columns:
    df_new_2[col] = df_new_2[col].astype('int')

In [70]:
df_new_2['NEIGHBORHOOD_CLUSTER'] = df_new_2['NEIGHBORHOOD_CLUSTER'].str.extract('(\d+)').astype('Int64')

In [71]:
df_new_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   X                     4164 non-null   float64            
 1   Y                     4164 non-null   float64            
 2   CCN                   4164 non-null   int64              
 3   REPORT_DAT            4164 non-null   datetime64[ns, UTC]
 4   SHIFT                 4164 non-null   category           
 5   METHOD                4164 non-null   category           
 6   OFFENSE               4164 non-null   category           
 7   BLOCK                 4164 non-null   object             
 8   XBLOCK                4164 non-null   float64            
 9   YBLOCK                4164 non-null   float64            
 10  WARD                  4164 non-null   int64              
 11  ANC                   4164 non-null   category           
 12  DISTRI

In [72]:
df_new_2.isnull().sum()

X                       0
Y                       0
CCN                     0
REPORT_DAT              0
SHIFT                   0
METHOD                  0
OFFENSE                 0
BLOCK                   0
XBLOCK                  0
YBLOCK                  0
WARD                    0
ANC                     0
DISTRICT                0
PSA                     0
NEIGHBORHOOD_CLUSTER    0
BLOCK_GROUP             0
CENSUS_TRACT            0
LATITUDE                0
LONGITUDE               0
START_DATE              0
END_DATE                0
OBJECTID                0
BLOCK_CODE              0
dtype: int64

In [73]:
df_new_2.drop(columns=['BLOCK_CODE'], inplace=True)

In [74]:
df_new_2.isnull().sum()   

X                       0
Y                       0
CCN                     0
REPORT_DAT              0
SHIFT                   0
METHOD                  0
OFFENSE                 0
BLOCK                   0
XBLOCK                  0
YBLOCK                  0
WARD                    0
ANC                     0
DISTRICT                0
PSA                     0
NEIGHBORHOOD_CLUSTER    0
BLOCK_GROUP             0
CENSUS_TRACT            0
LATITUDE                0
LONGITUDE               0
START_DATE              0
END_DATE                0
OBJECTID                0
dtype: int64

In [75]:
df_new_2.to_csv('data_crime_final.csv', index=False)

In [76]:
df_new_2.to_parquet('data_crime_final.parquet', engine='pyarrow')

<h3><b>ADD NEW COLUMN</b></h3> 

In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  
import seaborn as sns

df_new_3 = pd.read_parquet('data_crime_final.parquet')
df_new_3.head(10)

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,DISTRICT,PSA,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,LATITUDE,LONGITUDE,START_DATE,END_DATE,OBJECTID
0,400046.410,137753.6500,25017503,2025-02-05 19:50:58+00:00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.410000,137753.650000,...,5,501,23,2,8803,38.907642,-76.999465,2025-02-05 19:22:00+00:00,2025-02-05 19:25:00+00:00,685784661
1,397868.680,141140.3800,25013087,2025-01-28 12:08:29+00:00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.680000,141140.380000,...,4,404,18,2,2503,38.938148,-77.024583,2025-01-28 11:06:00+00:00,2025-01-28 11:07:00+00:00,685784675
2,399489.480,137372.6500,25012701,2025-01-27 17:28:06+00:00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.480000,137372.650000,...,5,501,25,3,10603,38.904210,-77.005886,2025-01-27 15:00:00+00:00,2025-01-27 15:20:00+00:00,685784796
3,401498.970,138734.5700,25008887,2025-01-19 19:59:51+00:00,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF NEW YORK AVENUE NE,401498.970000,138734.570000,...,5,506,23,1,8803,38.916477,-76.982716,2025-01-19 19:22:00+00:00,2025-01-19 19:32:00+00:00,685784829
4,400212.850,137949.3300,25011684,2025-01-25 15:45:47+00:00,DAY,OTHERS,THEFT F/AUTO,1300 - 1399 BLOCK OF 5TH STREET NE,400212.850000,137949.330000,...,5,501,23,2,8803,38.909405,-76.997546,2025-01-24 00:00:00+00:00,2025-01-24 03:30:00+00:00,685784830
5,397655.460,139302.9100,25013283,2025-01-28 20:29:39+00:00,EVENING,GUN,ROBBERY,2300 - 2499 BLOCK OF 11TH STREET NW,397655.460000,139302.910000,...,3,304,2,3,3600,38.921595,-77.027036,2025-01-28 17:50:00+00:00,2025-01-28 17:52:00+00:00,685784831
6,397921.520,138712.8100,25002549,2025-01-06 03:12:43+00:00,EVENING,OTHERS,THEFT/OTHER,1900 - 1999 BLOCK OF 9TH STREET NW,397921.520000,138712.810000,...,3,305,3,2,4402,38.916280,-77.023967,2025-01-05 11:00:00+00:00,2025-01-05 11:30:00+00:00,685784832
7,401627.937,132937.3269,25420547,2025-02-06 12:01:12+00:00,DAY,OTHERS,THEFT/OTHER,1600 - 1699 BLOCK OF V STREET SE,401627.936959,132937.326937,...,7,701,28,2,7504,38.864253,-76.981242,2025-02-05 21:31:00+00:00,2025-02-05 21:40:00+00:00,685784987
8,393619.540,143740.0900,25420388,2025-01-30 11:31:22+00:00,MIDNIGHT,OTHERS,THEFT/OTHER,5400 - 5419 BLOCK OF CONNECTICUT AVENUE NW,393619.540000,143740.090000,...,2,201,10,1,1100,38.961547,-77.073619,2024-12-23 15:00:00+00:00,2025-01-22 00:50:00+00:00,685785027
9,399868.920,143373.5400,25018642,2025-02-08 02:07:45+00:00,EVENING,OTHERS,THEFT/OTHER,310 - 399 BLOCK OF RIGGS ROAD NE,399868.920000,143373.540000,...,4,406,19,3,9505,38.958268,-77.001512,2025-02-07 22:34:00+00:00,2025-02-07 23:56:00+00:00,685785032


Add new columns: 'DATE(REPORT)', 'DATE(START)', 'DATE(END)' 

In [78]:
df_new_3['DATE(REPORT)'] = df_new_3['REPORT_DAT'].dt.date
df_new_3['DATE(START)'] = df_new_3['START_DATE'].dt.date
df_new_3['DATE(END)'] = df_new_3['END_DATE'].dt.date

In [79]:
df_new_3.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,LATITUDE,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END)
0,400046.41,137753.65,25017503,2025-02-05 19:50:58+00:00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.41,137753.65,...,2,8803,38.907642,-76.999465,2025-02-05 19:22:00+00:00,2025-02-05 19:25:00+00:00,685784661,2025-02-05,2025-02-05,2025-02-05
1,397868.68,141140.38,25013087,2025-01-28 12:08:29+00:00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.68,141140.38,...,2,2503,38.938148,-77.024583,2025-01-28 11:06:00+00:00,2025-01-28 11:07:00+00:00,685784675,2025-01-28,2025-01-28,2025-01-28
2,399489.48,137372.65,25012701,2025-01-27 17:28:06+00:00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.48,137372.65,...,3,10603,38.904210,-77.005886,2025-01-27 15:00:00+00:00,2025-01-27 15:20:00+00:00,685784796,2025-01-27,2025-01-27,2025-01-27
3,401498.97,138734.57,25008887,2025-01-19 19:59:51+00:00,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF NEW YORK AVENUE NE,401498.97,138734.57,...,1,8803,38.916477,-76.982716,2025-01-19 19:22:00+00:00,2025-01-19 19:32:00+00:00,685784829,2025-01-19,2025-01-19,2025-01-19
4,400212.85,137949.33,25011684,2025-01-25 15:45:47+00:00,DAY,OTHERS,THEFT F/AUTO,1300 - 1399 BLOCK OF 5TH STREET NE,400212.85,137949.33,...,2,8803,38.909405,-76.997546,2025-01-24 00:00:00+00:00,2025-01-24 03:30:00+00:00,685784830,2025-01-25,2025-01-24,2025-01-24


In [80]:
df_new_3['DATE(REPORT)'] = pd.to_datetime(df_new_3['DATE(REPORT)'])
df_new_3['DATE(START)'] = pd.to_datetime(df_new_3['DATE(START)'])
df_new_3['DATE(END)'] = pd.to_datetime(df_new_3['DATE(END)'])

Add new columns: 'TIME(REPORT)', 'TIME(START)', 'TIME(END)' 

In [81]:
df_new_3['TIME(REPORT)'] = df_new_3['REPORT_DAT'].dt.time
df_new_3['TIME(START)'] = df_new_3['START_DATE'].dt.time    
df_new_3['TIME(END)'] = df_new_3['END_DATE'].dt.time

In [82]:
df_new_3.head(10)

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END),TIME(REPORT),TIME(START),TIME(END)
0,400046.410,137753.6500,25017503,2025-02-05 19:50:58+00:00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.410000,137753.650000,...,-76.999465,2025-02-05 19:22:00+00:00,2025-02-05 19:25:00+00:00,685784661,2025-02-05,2025-02-05,2025-02-05,19:50:58,19:22:00,19:25:00
1,397868.680,141140.3800,25013087,2025-01-28 12:08:29+00:00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.680000,141140.380000,...,-77.024583,2025-01-28 11:06:00+00:00,2025-01-28 11:07:00+00:00,685784675,2025-01-28,2025-01-28,2025-01-28,12:08:29,11:06:00,11:07:00
2,399489.480,137372.6500,25012701,2025-01-27 17:28:06+00:00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.480000,137372.650000,...,-77.005886,2025-01-27 15:00:00+00:00,2025-01-27 15:20:00+00:00,685784796,2025-01-27,2025-01-27,2025-01-27,17:28:06,15:00:00,15:20:00
3,401498.970,138734.5700,25008887,2025-01-19 19:59:51+00:00,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF NEW YORK AVENUE NE,401498.970000,138734.570000,...,-76.982716,2025-01-19 19:22:00+00:00,2025-01-19 19:32:00+00:00,685784829,2025-01-19,2025-01-19,2025-01-19,19:59:51,19:22:00,19:32:00
4,400212.850,137949.3300,25011684,2025-01-25 15:45:47+00:00,DAY,OTHERS,THEFT F/AUTO,1300 - 1399 BLOCK OF 5TH STREET NE,400212.850000,137949.330000,...,-76.997546,2025-01-24 00:00:00+00:00,2025-01-24 03:30:00+00:00,685784830,2025-01-25,2025-01-24,2025-01-24,15:45:47,00:00:00,03:30:00
5,397655.460,139302.9100,25013283,2025-01-28 20:29:39+00:00,EVENING,GUN,ROBBERY,2300 - 2499 BLOCK OF 11TH STREET NW,397655.460000,139302.910000,...,-77.027036,2025-01-28 17:50:00+00:00,2025-01-28 17:52:00+00:00,685784831,2025-01-28,2025-01-28,2025-01-28,20:29:39,17:50:00,17:52:00
6,397921.520,138712.8100,25002549,2025-01-06 03:12:43+00:00,EVENING,OTHERS,THEFT/OTHER,1900 - 1999 BLOCK OF 9TH STREET NW,397921.520000,138712.810000,...,-77.023967,2025-01-05 11:00:00+00:00,2025-01-05 11:30:00+00:00,685784832,2025-01-06,2025-01-05,2025-01-05,03:12:43,11:00:00,11:30:00
7,401627.937,132937.3269,25420547,2025-02-06 12:01:12+00:00,DAY,OTHERS,THEFT/OTHER,1600 - 1699 BLOCK OF V STREET SE,401627.936959,132937.326937,...,-76.981242,2025-02-05 21:31:00+00:00,2025-02-05 21:40:00+00:00,685784987,2025-02-06,2025-02-05,2025-02-05,12:01:12,21:31:00,21:40:00
8,393619.540,143740.0900,25420388,2025-01-30 11:31:22+00:00,MIDNIGHT,OTHERS,THEFT/OTHER,5400 - 5419 BLOCK OF CONNECTICUT AVENUE NW,393619.540000,143740.090000,...,-77.073619,2024-12-23 15:00:00+00:00,2025-01-22 00:50:00+00:00,685785027,2025-01-30,2024-12-23,2025-01-22,11:31:22,15:00:00,00:50:00
9,399868.920,143373.5400,25018642,2025-02-08 02:07:45+00:00,EVENING,OTHERS,THEFT/OTHER,310 - 399 BLOCK OF RIGGS ROAD NE,399868.920000,143373.540000,...,-77.001512,2025-02-07 22:34:00+00:00,2025-02-07 23:56:00+00:00,685785032,2025-02-08,2025-02-07,2025-02-07,02:07:45,22:34:00,23:56:00


In [83]:
df_new_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   X                     4164 non-null   float64            
 1   Y                     4164 non-null   float64            
 2   CCN                   4164 non-null   int64              
 3   REPORT_DAT            4164 non-null   datetime64[ns, UTC]
 4   SHIFT                 4164 non-null   category           
 5   METHOD                4164 non-null   category           
 6   OFFENSE               4164 non-null   category           
 7   BLOCK                 4164 non-null   object             
 8   XBLOCK                4164 non-null   float64            
 9   YBLOCK                4164 non-null   float64            
 10  WARD                  4164 non-null   int64              
 11  ANC                   4164 non-null   category           
 12  DISTRI

In [84]:
time_column = ['TIME(REPORT)', 'TIME(START)', 'TIME(END)']
for col in time_column:
    df_new_3[col] = pd.to_timedelta(df_new_3[col].astype(str))

In [85]:
df_new_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   X                     4164 non-null   float64            
 1   Y                     4164 non-null   float64            
 2   CCN                   4164 non-null   int64              
 3   REPORT_DAT            4164 non-null   datetime64[ns, UTC]
 4   SHIFT                 4164 non-null   category           
 5   METHOD                4164 non-null   category           
 6   OFFENSE               4164 non-null   category           
 7   BLOCK                 4164 non-null   object             
 8   XBLOCK                4164 non-null   float64            
 9   YBLOCK                4164 non-null   float64            
 10  WARD                  4164 non-null   int64              
 11  ANC                   4164 non-null   category           
 12  DISTRI

Saving final result

In [86]:
df_new_3.to_csv('data_final.csv', index=False)

In [87]:
df_new_3.to_parquet('data_final.parquet', engine='pyarrow')

<h3><b>DATA ANALYSIS</b></h3> 

**Crime data**

In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  
import seaborn as sns

data = pd.read_parquet('data_final.parquet')
data.head(10)

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END),TIME(REPORT),TIME(START),TIME(END)
0,400046.410,137753.6500,25017503,2025-02-05 19:50:58+00:00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.410000,137753.650000,...,-76.999465,2025-02-05 19:22:00+00:00,2025-02-05 19:25:00+00:00,685784661,2025-02-05,2025-02-05,2025-02-05,0 days 19:50:58,0 days 19:22:00,0 days 19:25:00
1,397868.680,141140.3800,25013087,2025-01-28 12:08:29+00:00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.680000,141140.380000,...,-77.024583,2025-01-28 11:06:00+00:00,2025-01-28 11:07:00+00:00,685784675,2025-01-28,2025-01-28,2025-01-28,0 days 12:08:29,0 days 11:06:00,0 days 11:07:00
2,399489.480,137372.6500,25012701,2025-01-27 17:28:06+00:00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.480000,137372.650000,...,-77.005886,2025-01-27 15:00:00+00:00,2025-01-27 15:20:00+00:00,685784796,2025-01-27,2025-01-27,2025-01-27,0 days 17:28:06,0 days 15:00:00,0 days 15:20:00
3,401498.970,138734.5700,25008887,2025-01-19 19:59:51+00:00,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF NEW YORK AVENUE NE,401498.970000,138734.570000,...,-76.982716,2025-01-19 19:22:00+00:00,2025-01-19 19:32:00+00:00,685784829,2025-01-19,2025-01-19,2025-01-19,0 days 19:59:51,0 days 19:22:00,0 days 19:32:00
4,400212.850,137949.3300,25011684,2025-01-25 15:45:47+00:00,DAY,OTHERS,THEFT F/AUTO,1300 - 1399 BLOCK OF 5TH STREET NE,400212.850000,137949.330000,...,-76.997546,2025-01-24 00:00:00+00:00,2025-01-24 03:30:00+00:00,685784830,2025-01-25,2025-01-24,2025-01-24,0 days 15:45:47,0 days 00:00:00,0 days 03:30:00
5,397655.460,139302.9100,25013283,2025-01-28 20:29:39+00:00,EVENING,GUN,ROBBERY,2300 - 2499 BLOCK OF 11TH STREET NW,397655.460000,139302.910000,...,-77.027036,2025-01-28 17:50:00+00:00,2025-01-28 17:52:00+00:00,685784831,2025-01-28,2025-01-28,2025-01-28,0 days 20:29:39,0 days 17:50:00,0 days 17:52:00
6,397921.520,138712.8100,25002549,2025-01-06 03:12:43+00:00,EVENING,OTHERS,THEFT/OTHER,1900 - 1999 BLOCK OF 9TH STREET NW,397921.520000,138712.810000,...,-77.023967,2025-01-05 11:00:00+00:00,2025-01-05 11:30:00+00:00,685784832,2025-01-06,2025-01-05,2025-01-05,0 days 03:12:43,0 days 11:00:00,0 days 11:30:00
7,401627.937,132937.3269,25420547,2025-02-06 12:01:12+00:00,DAY,OTHERS,THEFT/OTHER,1600 - 1699 BLOCK OF V STREET SE,401627.936959,132937.326937,...,-76.981242,2025-02-05 21:31:00+00:00,2025-02-05 21:40:00+00:00,685784987,2025-02-06,2025-02-05,2025-02-05,0 days 12:01:12,0 days 21:31:00,0 days 21:40:00
8,393619.540,143740.0900,25420388,2025-01-30 11:31:22+00:00,MIDNIGHT,OTHERS,THEFT/OTHER,5400 - 5419 BLOCK OF CONNECTICUT AVENUE NW,393619.540000,143740.090000,...,-77.073619,2024-12-23 15:00:00+00:00,2025-01-22 00:50:00+00:00,685785027,2025-01-30,2024-12-23,2025-01-22,0 days 11:31:22,0 days 15:00:00,0 days 00:50:00
9,399868.920,143373.5400,25018642,2025-02-08 02:07:45+00:00,EVENING,OTHERS,THEFT/OTHER,310 - 399 BLOCK OF RIGGS ROAD NE,399868.920000,143373.540000,...,-77.001512,2025-02-07 22:34:00+00:00,2025-02-07 23:56:00+00:00,685785032,2025-02-08,2025-02-07,2025-02-07,0 days 02:07:45,0 days 22:34:00,0 days 23:56:00


**Total cases analysis**

In [89]:
duplicate_rows = data[data.duplicated()]

if not duplicate_rows.empty:
    print('Duplicate Rows based on all columns:')
    print(duplicate_rows)
else:
    print('No Duplicate Rows')

No Duplicate Rows


In [90]:
total_cases = len(data)
print(f"Total number of cases: {total_cases}")

Total number of cases: 4164


In [91]:
default_date = "1989-05-19"

earliest_date = data.loc[data['DATE(START)'] != default_date, 'DATE(START)'].min()
latest_date = data['DATE(START)'].max()

print('Date crime occured: ')
print(f"Earliest date: {earliest_date}")
print(f"Latest date: {latest_date}")


Date crime occured: 
Earliest date: 2005-09-27 00:00:00
Latest date: 2025-03-04 00:00:00


In [92]:
majority_case = data['DATE(START)'].mode()
print(f"Most common date: {majority_case}")


Most common date: 0   2025-01-25
Name: DATE(START), dtype: datetime64[ns]


There's an invalid case. When majority of cases occured/retreived in last 2024 and 2 months early in 2025 , this case in month 9/2005 
(There may be recording errors)

In [93]:
data[data['DATE(START)'].dt.year == 2005].head() 

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END),TIME(REPORT),TIME(START),TIME(END)
525,400588.465,129470.0424,5131706,2025-02-05 05:00:00+00:00,MIDNIGHT,GUN,HOMICIDE,800 - 899 BLOCK OF XENIA STREET SE,400588.46499,129470.042358,...,-76.993222,2005-09-27 07:25:00+00:00,2005-09-27 07:26:00+00:00,686110728,2025-02-05,2005-09-27,2005-09-27,0 days 05:00:00,0 days 07:25:00,0 days 07:26:00


In [94]:
data.drop(index=525, inplace=True)

**Total case analysis**

In [95]:
print(f'Total cases: {len(data)}')

Total cases: 4163


In [96]:
default_date = "1989-05-19"

earliest_date = data.loc[data['DATE(START)'] != default_date, 'DATE(START)'].min()
latest_date = data['DATE(START)'].max()

print('Date crime occured: ')
print(f"Earliest date: {earliest_date}")
print(f"Latest date: {latest_date}")

Date crime occured: 
Earliest date: 2022-07-30 00:00:00
Latest date: 2025-03-04 00:00:00


There's an invalid case. When majority of cases occured/retreived in last 2024 and 2 months early in 2025, this case in month 7/2022
(There may be recording errors)

In [97]:
data[data['DATE(START)'].dt.year == 2022].head() 

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END),TIME(REPORT),TIME(START),TIME(END)
2677,399075.71,128801.54,22108667,2025-01-02 05:00:00+00:00,MIDNIGHT,GUN,HOMICIDE,1 - 199 BLOCK OF DARRINGTON STREET SW,399075.71,128801.54,...,-77.010645,2022-07-30 01:12:00+00:00,2022-07-30 01:13:00+00:00,686284217,2025-01-02,2022-07-30,2022-07-30,0 days 05:00:00,0 days 01:12:00,0 days 01:13:00


In [98]:
data.drop(index=2677, inplace=True)

**Total case analysis**

In [99]:
print(f'Total cases: {len(data)}')

Total cases: 4162


In [100]:
default_date = "1989-05-19"

earliest_date = data.loc[data['DATE(START)'] != default_date, 'DATE(START)'].min()
latest_date = data['DATE(START)'].max()

print('Date crime occured: ')
print(f"Earliest date: {earliest_date}")
print(f"Latest date: {latest_date}")

Date crime occured: 
Earliest date: 2024-04-15 00:00:00
Latest date: 2025-03-04 00:00:00


In [101]:
year = 2024
month = 12 

data_2024 = data[data['DATE(START)'].dt.year == year]

for i in range(1, month + 1):
    total_cases = data_2024[data_2024['DATE(START)'].dt.month == i].shape[0]
    print(f"Total cases in {year}, Month {i}: {total_cases}")


Total cases in 2024, Month 1: 0
Total cases in 2024, Month 2: 0
Total cases in 2024, Month 3: 0
Total cases in 2024, Month 4: 2
Total cases in 2024, Month 5: 0
Total cases in 2024, Month 6: 0
Total cases in 2024, Month 7: 0
Total cases in 2024, Month 8: 2
Total cases in 2024, Month 9: 3
Total cases in 2024, Month 10: 4
Total cases in 2024, Month 11: 15
Total cases in 2024, Month 12: 130


In [102]:
data_apr_2024 = data[(data['DATE(START)'].dt.year == 2024) & (data['DATE(START)'].dt.month == 4)]
data_apr_2024.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END),TIME(REPORT),TIME(START),TIME(END)
950,395423.62,138083.38,25024960,2025-02-20 19:17:19+00:00,DAY,OTHERS,THEFT/OTHER,2300 - 2699 BLOCK OF Q STREET NW,395423.62,138083.38,...,-77.052765,2024-04-18 12:10:00+00:00,2024-04-20 15:10:00+00:00,686120602,2025-02-20,2024-04-18,2024-04-20,0 days 19:17:19,0 days 12:10:00,0 days 15:10:00
3616,399878.21,137742.29,25027260,2025-02-25 03:47:32+00:00,EVENING,OTHERS,SEX ABUSE,300 - 385 BLOCK OF FLORIDA AVENUE NE,399878.21,137742.29,...,-77.001404,2024-04-15 23:00:00+00:00,2024-04-15 23:02:00+00:00,686320947,2025-02-25,2024-04-15,2024-04-15,0 days 03:47:32,0 days 23:00:00,0 days 23:02:00


There's an invalid case (for analyisis context). When majority of cases occured/retreived in last 2024 and 2 months early in 2025, this case in month 4 (there is a period of recording that does not match or make sense because of the gap between months 4 to 8, where in months 5,6,7 there are no cases/case reports at all and this is very odd in the context of the dataset (in reality, it is impossible that there are no crime reports in these months, unless the crime reports are not recorded in the report/system, or because the latest data is available only for the end of 2024 to the beginning of 2025)).

In [103]:
data.drop(index=950, inplace=True)
data.drop(index=3616, inplace=True)

In [104]:
year = 2024
month = 12 

data_2024 = data[data['DATE(START)'].dt.year == year]

for i in range(1, month + 1):
    total_cases = data_2024[data_2024['DATE(START)'].dt.month == i].shape[0]
    print(f"Total cases in {year}, Month {i}: {total_cases}")

Total cases in 2024, Month 1: 0
Total cases in 2024, Month 2: 0
Total cases in 2024, Month 3: 0
Total cases in 2024, Month 4: 0
Total cases in 2024, Month 5: 0
Total cases in 2024, Month 6: 0
Total cases in 2024, Month 7: 0
Total cases in 2024, Month 8: 2
Total cases in 2024, Month 9: 3
Total cases in 2024, Month 10: 4
Total cases in 2024, Month 11: 15
Total cases in 2024, Month 12: 130


**Total cases (valid)**

In [105]:
data.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END),TIME(REPORT),TIME(START),TIME(END)
0,400046.41,137753.65,25017503,2025-02-05 19:50:58+00:00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.41,137753.65,...,-76.999465,2025-02-05 19:22:00+00:00,2025-02-05 19:25:00+00:00,685784661,2025-02-05,2025-02-05,2025-02-05,0 days 19:50:58,0 days 19:22:00,0 days 19:25:00
1,397868.68,141140.38,25013087,2025-01-28 12:08:29+00:00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.68,141140.38,...,-77.024583,2025-01-28 11:06:00+00:00,2025-01-28 11:07:00+00:00,685784675,2025-01-28,2025-01-28,2025-01-28,0 days 12:08:29,0 days 11:06:00,0 days 11:07:00
2,399489.48,137372.65,25012701,2025-01-27 17:28:06+00:00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.48,137372.65,...,-77.005886,2025-01-27 15:00:00+00:00,2025-01-27 15:20:00+00:00,685784796,2025-01-27,2025-01-27,2025-01-27,0 days 17:28:06,0 days 15:00:00,0 days 15:20:00
3,401498.97,138734.57,25008887,2025-01-19 19:59:51+00:00,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF NEW YORK AVENUE NE,401498.97,138734.57,...,-76.982716,2025-01-19 19:22:00+00:00,2025-01-19 19:32:00+00:00,685784829,2025-01-19,2025-01-19,2025-01-19,0 days 19:59:51,0 days 19:22:00,0 days 19:32:00
4,400212.85,137949.33,25011684,2025-01-25 15:45:47+00:00,DAY,OTHERS,THEFT F/AUTO,1300 - 1399 BLOCK OF 5TH STREET NE,400212.85,137949.33,...,-76.997546,2025-01-24 00:00:00+00:00,2025-01-24 03:30:00+00:00,685784830,2025-01-25,2025-01-24,2025-01-24,0 days 15:45:47,0 days 00:00:00,0 days 03:30:00


In [106]:
print(f'Total cases: {len(data)}')

Total cases: 4160


In [107]:
earliest_case = data.loc[data['DATE(START)'] != default_date, 'DATE(START)'].min()
latest_case = data['DATE(START)'].max()

print('======== Date crime occured ========')
print(f"Earliest case: {earliest_case}")
print(f"Latest case: {latest_case}")

======== Date crime occured ========
Earliest case: 2024-08-06 00:00:00
Latest case: 2025-03-04 00:00:00


Decision: in the period from the end of 2024 to the beginning of March 2025, the total number of reported crimes reached 4160.

In [108]:
data['DISTRICT'] = data['DISTRICT'].astype(int)

In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4160 entries, 0 to 4163
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   X                     4160 non-null   float64            
 1   Y                     4160 non-null   float64            
 2   CCN                   4160 non-null   int64              
 3   REPORT_DAT            4160 non-null   datetime64[ns, UTC]
 4   SHIFT                 4160 non-null   category           
 5   METHOD                4160 non-null   category           
 6   OFFENSE               4160 non-null   category           
 7   BLOCK                 4160 non-null   object             
 8   XBLOCK                4160 non-null   float64            
 9   YBLOCK                4160 non-null   float64            
 10  WARD                  4160 non-null   int64              
 11  ANC                   4160 non-null   category           
 12  DISTRICT   

Ready to analysis with tableau

In [110]:
data.to_csv('CRIME_DATA.csv', index=False)

In [111]:
data.to_parquet('CRIME_DATA.parquet', engine='pyarrow')

<h3><b>CLEANING DATA VALUE PART 2</b></h3> 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

datasets = pd.read_parquet('CRIME_DATA.parquet')
datasets.head(10)


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END),TIME(REPORT),TIME(START),TIME(END)
0,400046.410,137753.6500,25017503,2025-02-05 19:50:58+00:00,DAY,OTHERS,MOTOR VEHICLE THEFT,400 - 499 BLOCK OF MORSE STREET NE,400046.410000,137753.650000,...,-76.999465,2025-02-05 19:22:00+00:00,2025-02-05 19:25:00+00:00,685784661,2025-02-05,2025-02-05,2025-02-05,0 days 19:50:58,0 days 19:22:00,0 days 19:25:00
1,397868.680,141140.3800,25013087,2025-01-28 12:08:29+00:00,DAY,OTHERS,THEFT/OTHER,3800 - 3899 BLOCK OF GEORGIA AVENUE NW,397868.680000,141140.380000,...,-77.024583,2025-01-28 11:06:00+00:00,2025-01-28 11:07:00+00:00,685784675,2025-01-28,2025-01-28,2025-01-28,0 days 12:08:29,0 days 11:06:00,0 days 11:07:00
2,399489.480,137372.6500,25012701,2025-01-27 17:28:06+00:00,DAY,OTHERS,ROBBERY,1100 - 1150 BLOCK OF 1ST STREET NE,399489.480000,137372.650000,...,-77.005886,2025-01-27 15:00:00+00:00,2025-01-27 15:20:00+00:00,685784796,2025-01-27,2025-01-27,2025-01-27,0 days 17:28:06,0 days 15:00:00,0 days 15:20:00
3,401498.970,138734.5700,25008887,2025-01-19 19:59:51+00:00,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF NEW YORK AVENUE NE,401498.970000,138734.570000,...,-76.982716,2025-01-19 19:22:00+00:00,2025-01-19 19:32:00+00:00,685784829,2025-01-19,2025-01-19,2025-01-19,0 days 19:59:51,0 days 19:22:00,0 days 19:32:00
4,400212.850,137949.3300,25011684,2025-01-25 15:45:47+00:00,DAY,OTHERS,THEFT F/AUTO,1300 - 1399 BLOCK OF 5TH STREET NE,400212.850000,137949.330000,...,-76.997546,2025-01-24 00:00:00+00:00,2025-01-24 03:30:00+00:00,685784830,2025-01-25,2025-01-24,2025-01-24,0 days 15:45:47,0 days 00:00:00,0 days 03:30:00
5,397655.460,139302.9100,25013283,2025-01-28 20:29:39+00:00,EVENING,GUN,ROBBERY,2300 - 2499 BLOCK OF 11TH STREET NW,397655.460000,139302.910000,...,-77.027036,2025-01-28 17:50:00+00:00,2025-01-28 17:52:00+00:00,685784831,2025-01-28,2025-01-28,2025-01-28,0 days 20:29:39,0 days 17:50:00,0 days 17:52:00
6,397921.520,138712.8100,25002549,2025-01-06 03:12:43+00:00,EVENING,OTHERS,THEFT/OTHER,1900 - 1999 BLOCK OF 9TH STREET NW,397921.520000,138712.810000,...,-77.023967,2025-01-05 11:00:00+00:00,2025-01-05 11:30:00+00:00,685784832,2025-01-06,2025-01-05,2025-01-05,0 days 03:12:43,0 days 11:00:00,0 days 11:30:00
7,401627.937,132937.3269,25420547,2025-02-06 12:01:12+00:00,DAY,OTHERS,THEFT/OTHER,1600 - 1699 BLOCK OF V STREET SE,401627.936959,132937.326937,...,-76.981242,2025-02-05 21:31:00+00:00,2025-02-05 21:40:00+00:00,685784987,2025-02-06,2025-02-05,2025-02-05,0 days 12:01:12,0 days 21:31:00,0 days 21:40:00
8,393619.540,143740.0900,25420388,2025-01-30 11:31:22+00:00,MIDNIGHT,OTHERS,THEFT/OTHER,5400 - 5419 BLOCK OF CONNECTICUT AVENUE NW,393619.540000,143740.090000,...,-77.073619,2024-12-23 15:00:00+00:00,2025-01-22 00:50:00+00:00,685785027,2025-01-30,2024-12-23,2025-01-22,0 days 11:31:22,0 days 15:00:00,0 days 00:50:00
9,399868.920,143373.5400,25018642,2025-02-08 02:07:45+00:00,EVENING,OTHERS,THEFT/OTHER,310 - 399 BLOCK OF RIGGS ROAD NE,399868.920000,143373.540000,...,-77.001512,2025-02-07 22:34:00+00:00,2025-02-07 23:56:00+00:00,685785032,2025-02-08,2025-02-07,2025-02-07,0 days 02:07:45,0 days 22:34:00,0 days 23:56:00


In [3]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4160 entries, 0 to 4163
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   X                     4160 non-null   float64            
 1   Y                     4160 non-null   float64            
 2   CCN                   4160 non-null   int64              
 3   REPORT_DAT            4160 non-null   datetime64[ns, UTC]
 4   SHIFT                 4160 non-null   category           
 5   METHOD                4160 non-null   category           
 6   OFFENSE               4160 non-null   category           
 7   BLOCK                 4160 non-null   object             
 8   XBLOCK                4160 non-null   float64            
 9   YBLOCK                4160 non-null   float64            
 10  WARD                  4160 non-null   int64              
 11  ANC                   4160 non-null   category           
 12  DISTRICT   

Crime Trend

In [14]:
default_date = "1989-05-19"
count_default = (datasets['DATE(END)'] == default_date).sum()
print(f"Count of default date: {count_default}")

Count of default date: 1


In [ ]:
default_date = pd.to_datetime("1989-05-19")
default_date_rows = datasets[datasets['DATE(END)'] == default_date]
default_date_rows

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,LONGITUDE,START_DATE,END_DATE,OBJECTID,DATE(REPORT),DATE(START),DATE(END),TIME(REPORT),TIME(START),TIME(END)
4058,400425.0658,132906.2884,89272849,2025-01-15 05:00:00+00:00,MIDNIGHT,GUN,HOMICIDE,INTERSTATE 295 SOUTHBOUND AND RAMP TO HOWARD R...,400425.065813,132906.288375,...,-76.995102,1989-05-19 21:10:00+00:00,1989-05-19 21:11:00+00:00,686344406,2025-01-15,1989-05-19,1989-05-19,0 days 05:00:00,0 days 21:10:00,0 days 21:11:00


In [ ]:
default_date = pd.to_datetime("1989-05-19")
default_date_rows = datasets[datasets['DATE(END)'] == default_date].index
datasets = datasets.drop(default_date_rows)
print(len(datasets))

4159


In [24]:
datasets.to_parquet('CRIME_DATA_FINAL.parquet', engine='pyarrow')

In [25]:
datasets.to_csv('CRIME_DATA_FINAL.csv', index = False)